# Part 4, Topic 2: CPA on Firmware Implementation of AES (HARDWARE)

---
**THIS IS NOT THE COMPLETE TUTORIAL - see file with `(MAIN)` in the name.**

---

First you'll need to select which hardware setup you have. You'll need to select a `SCOPETYPE`, a `PLATFORM`, and a `CRYPTO_TARGET`. `SCOPETYPE` can either be `'OPENADC'` for the CWLite/CW1200 or `'CWNANO'` for the CWNano. `PLATFORM` is the target device, with `'CWLITEARM'`/`'CW308_STM32F3'` being the best supported option, followed by `'CWLITEXMEGA'`/`'CW308_XMEGA'`, then by `'CWNANO'`. `CRYPTO_TARGET` selects the crypto implementation, with `'TINYAES128C'` working on all platforms. An alternative for `'CWLITEXMEGA'` targets is `'AVRCRYPTOLIB'`. For example:

```python
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
CRYPTO_TARGET='TINYAES128C'
SS_VER='SS_VER_2_1'
```

In [1]:
SCOPETYPE = 'CWNANO'
PLATFORM = 'CWNANO'
CRYPTO_TARGET = 'TINYAES128C'
SS_VER='SS_VER_1_1' #2_1 n'a pas marché tres bien pour le CWNano

The following code will build the firmware for the target.

In [2]:
%run "../../../Setup_Scripts/Setup_Generic.ipynb"

INFO: Found ChipWhisperer😍


In [3]:
cw.scope_logger.setLevel(cw.logging.DEBUG)

In [6]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
cd ../../../../hardware/victims/firmware/cpa_ops
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3 -j

Building for platform CWNANO with CRYPTO_TARGET=TINYAES128C
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
Blank crypto options, building for AES128
Building for platform CWNANO with CRYPTO_TARGET=TINYAES128C
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
Blank crypto options, building for AES128
make[1]: '.dep' is up to date.
Building for platform CWNANO with CRYPTO_TARGET=TINYAES128C
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
Blank crypto options, building for AES128
.
Welcome to another exciting ChipWhisperer target build!!
arm-none-eabi-gcc.exe (GNU Arm Embedded Toolchain 10-2020-q4-major) 10.2.1 20201103 (release)
Copyright (C) 2020 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Assembling: .././hal/stm32f0/stm32f0_startup.S
arm-none-eabi-gcc -c -mcpu=cortex-m0 -I. -x assembler-with-cpp -mthumb -mfloat-abi=soft -ffunction-sections -

C:\Users\diogenem\AppData\Local\Temp\ccKJbexc.s: Assembler messages:
C:\Users\diogenem\AppData\Local\Temp\ccKJbexc.s:218: Error: unshifted register required -- `and r7,r7,#255'
make[1]: *** [.././Makefile.inc:495: objdir-CWNANO/simpleserial-aes.o] Error 1
make[1]: *** Waiting for unfinished jobs....


    simpleserial-aes.c ....
Compiling:
    .././simpleserial/simpleserial.c ...Done!
.
Compiling:
    .././crypto/aes-independant.c ...Done!
.
Compiling:
    .././crypto/tiny-AES128-C/aes.c ...Done!
.
Compiling:
    .././hal/stm32f0_nano/stm32f0_hal_nano.c ...Done!
.
Compiling:
    .././hal/stm32f0/stm32f0_hal_lowlevel.c ...Done!


make: *** [.././Makefile.inc:338: all] Error 2


CalledProcessError: Command 'b'cd ../../../../hardware/victims/firmware/cpa_ops\nmake PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3 -j\n'' returned non-zero exit status 2.

In [ ]:
cw.program_target(scope, prog, "../../../../hardware/victims/firmware/cpa_ops/simpleserial-aes-{}.hex".format(PLATFORM))

We only need 50 traces this time to break AES!

In [ ]:
splot = cw.StreamPlot()
splot.plot()

In [17]:
from tqdm.notebook import trange
import numpy as np
import time

ktp = cw.ktp.Basic()
trace_array = []
textin_array = []

key, text = ktp.next()

target.set_key(key)

N = 50 #increase nano reliability
if PLATFORM=="CWNANO":
    N = 1000
for i in trange(N, desc='Capturing traces'):
    scope.arm()
    
    target.simpleserial_write('p', text)
    
    ret = scope.capture()
    if ret:
        print("Target timed out!")
        continue
    
    response = target.simpleserial_read('r', 16)
    
    trace_array.append(scope.get_last_trace())
    textin_array.append(text)
    
    key, text = ktp.next()
    splot.update(scope.get_last_trace())
    
trace_array = np.array(trace_array)

NameError: name 'cw' is not defined

We don't need the hardware anymore, so we'll disconnect:

In [4]:
#scope.dis()
#target.dis()